In [30]:
import os
import re
import importlib
import time

from datetime import datetime

In [31]:
import math

def degree_to_radian(degrees):
	'''Converts degrees (in decimal) to radians.'''
	return (math.pi/180)*degrees

def radian_to_nm(radians):
	'''Converts a distance in radians to a distance in nautical miles.'''
	return ((180*60)/math.pi)*radians

def calc_dist(latitude1,longitude1,latitude2,longitude2):
	'''Calculates and returns a distance in nautical miles given two lat/lon coordinate values.
		Follows the specification described in the Aviation Formulary v1.46
		by Ed Williams (originally at http://williams.best.vwh.net/avform.htm)
	'''
	lat1 = degree_to_radian(latitude1)
	lon1 = degree_to_radian(longitude1)
	lat2 = degree_to_radian(latitude2)
	lon2 = degree_to_radian(longitude2)

	# there are two implementations of this function.
	# implementation #1:
	#dist_rad = math.acos(math.sin(lat1) * math.sin(lat2) 
	#                   + math.cos(lat1) * math.cos(lat2) * math.cos(lon1-lon2))

	# implementation #2: (less subject to numerical error for short distances)
	dist_rad=2*math.asin(math.sqrt((math.sin((lat1-lat2)/2))**2 +
			   math.cos(lat1)*math.cos(lat2)*(math.sin((lon1-lon2)/2))**2))

	return radian_to_nm(dist_rad)

In [32]:

filename = 'Agadir'

filepath = os.path.join("cfg", filename + '.py')

instance = importlib.import_module("cfg." + filename)

start_time = time.time()

In [ ]:
print ("reading '" + instance.DIR + instance.INPUT + "'")

file = open(instance.DIR + "/" + instance.INPUT, "r")
elevation_data = file.read()
file.close()

ncols = int(re.search("ncols (.*)", elevation_data).group(1))
print ("number of columns:",ncols)

nrows = int(re.search("nrows (.*)", elevation_data).group(1))
print ("number of rows:", nrows)

latitude = float(re.search("xllcorner (.*)", elevation_data).group(1))
longitude = float(re.search("yllcorner (.*)", elevation_data).group(1))
data_delta = float(re.search("cellsize (.*)", elevation_data).group(1))

elevation_data = re.split("\n+", elevation_data)[6:-1]

print(len(elevation_data))

if (nrows != len(elevation_data)+1):

	print ("something wrong here")
	#quit()

map = {}
ocean = {}
min_depth = -11022.0
max_depth = 0.0

for i, line_str in enumerate(elevation_data[(nrows - 100 - instance.Y):nrows - 100]):

	line_dict = {}
	
	for j, element in enumerate(re.split("\s+", line_str)[:instance.X]):

		# if needed, here has to be the code for avg of depths

		if element < 0:

			ocean[i,j] = 1

			if element > min_depth and element < 0:

				min_depth = element

			if element < max_depth:

				max_depth = element

			line_dict[j] = element

	map[i] = line_dict

In [ ]:
map

In [ ]:

print (f"the total time spent is {(time.time()-start_time):.0f} seconds")

In [ ]:
from math import *

In [ ]:
def d(x0, y0, x1, y1):
	return sqrt((x0-x1)**2 + (y0-y1)**2)

In [ ]:
def g(alpha, instance):
	for i in range(len(instance.TS)-1):
		w_i = cos(instance.TS[i][0]/180.0*pi)
		w_ip1 = cos(instance.TS[i+1][0]/180.0*pi)
		s_i = instance.TS[i][1]
		s_ip1 = instance.TS[i+1][1]

		if w_i >= alpha and alpha >= w_ip1:
			return ( s_i + ( (s_ip1 - s_i) * (alpha - w_i) ) / ( w_ip1 - w_i ) )
		elif -w_i <= alpha and alpha <= -w_ip1:
			return ( s_i + ( (s_ip1 - s_i) * (alpha + w_i) ) / ( w_i - w_ip1 ) )

	return 0

In [ ]:
def check_line(x0, y0, x1, y1, map):
	dx = abs(x1-x0)
	if x0<x1:
		sx = 1
	else:
		sx = -1
	dy = -abs(y1-y0)
	if y0<y1:
		sy = 1
	else:
		sy = -1
	err = dx+dy

	while (1):
		if map[x0][y0] >= 0.0:
			return 1
		if x0==x1 and y0==y1:
			return None
		e2 = 2*err
		if e2 > dy:
			err = err + dy
			x0 = x0 + sx
		if e2 < dx:
			err = err + dx
			y0 = y0 + sy


In [ ]:
print ("Computing coverage")

detection_prob = {}

start_time_coverage = time.time()

if len(instance.TS) == 0: # without TS

	for tar_x, tar_y in ocean: # target

		for tx_x, tx_y in ocean: # source

			for rx_x, rx_y in ocean: # receiver

				# no obstacles between source-target and target-receiver, and source-reiver	
				if check_line(tx_x, tx_y, tar_x, tar_y, map) == None and check_line(tar_x, tar_y, rx_x, rx_y, map) == None:

					if instance.CC == 0: # probabilistic model
						
						if (((d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y)) / (instance.rho_0**2) - 1) / instance.b1) < e+10: # avoid numerical trouble from powers of large numbers
							aux = instance.pmax * (1 / (1 + 10**(((d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y)) / (instance.rho_0**2) - 1) / instance.b1))) * (1 / (1 + 10**((1 - (d(tx_x, tx_y, tar_x, tar_y) + d(rx_x, rx_y, tar_x, tar_y)) / (d(tx_x, tx_y, rx_x, rx_y) + 2*instance.rb)) / instance.b2)))
							
							if aux > instance.pmin:
								
								detection_prob[tar_x,tar_y,0,tx_x,tx_y,rx_x,rx_y] = log(1 - aux) # detection probabilitar_y

					else: # cookie-cutter model

						if d(tx_x, tx_y, tar_x, tar_y) * d(rx_x, rx_y, tar_x, tar_y) <= instance.rho_0**2 and d(tx_x, tx_y, tar_x, tar_y) + d(rx_x, rx_y, tar_x, tar_y) >= d(tx_x, tx_y, rx_x, rx_y) + 2*instance.rb: # check for inside range-of-day Cassini oval and outside direct-blast-effect
							
							detection_prob[tar_x, tar_y, 0, tx_x, tx_y, rx_x, rx_y] = 1 # sure detection

else: # with TS

	# only compute the angle we need to compute???

	for tar_x, tar_y in ocean: # target

		for tx_x, tx_y in ocean: # source

			if (tx_x,tx_y) != (tar_x,tar_y): # exclude equalitar_y of source and target (direct blast effect)
				
				for rx_x, rx_y in ocean: # receiver

					if (rx_x,rx_y) != (tar_x,tar_y): # exclude equalitar_y of receiver and target (direct blast effect)
					
						# no obstacles between source-target and target-receiver, and source-reiver
						if check_line(tx_x, tx_y, tar_x, tar_y, map) == None and check_line(tar_x, tar_y, rx_x, rx_y, map) == None:

							sqrt_tx_tar = 0.5 / ( sqrt((tx_x-tar_x)**2 + (tx_y-tar_y)**2) )
							sqrt_rx_tar = 0.5 / ( sqrt((rx_x-tar_x)**2 + (rx_y-tar_y)**2) )

							for theta in range(0, 180, instance.STEPS): # target angle

								my_theta = theta / 180.0 * pi
								my_sin_theta = sin(my_theta)
								my_cos_theta = cos(my_theta)

								if instance.CC == 0: # probabilistic model

									alpha = ( ((tx_x-tar_x)*my_cos_theta + (tx_y-tar_y)*my_sin_theta ) * sqrt_tx_tar + ((rx_x-tar_x)*my_cos_theta + (rx_y-tar_y)*my_sin_theta ) * sqrt_rx_tar )

									if (((d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y)) / ((instance.rho_0 + g(alpha, instance))**2) - 1)/instance.b1) < e+10: # avoid numerical trouble from powers of large numbers
										
										aux = instance.pmax * (1 / (1 + 10**(((d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y)) / ((instance.rho_0 + g(alpha, instance))**2) - 1)/instance.b1))) * (1 / (1 + 10**((1 - (d(tx_x,tx_y,tar_x,tar_y) + d(rx_x,rx_y,tar_x,tar_y)) / (d(tx_x,tx_y,rx_x,rx_y) + 2*instance.rb))/instance.b2)))
										
										if aux > instance.pmin:
											
											detection_prob[tar_x,tar_y,theta,tx_x,tx_y,rx_x,rx_y] = log(1 - aux) # detection probabilitar_y

								else: # cookie-cutter model
									
									if d(tx_x,tx_y,tar_x,tar_y) + d(rx_x,rx_y,tar_x,tar_y) >= d(tx_x,tx_y,rx_x,rx_y) + 2*instance.rb: # check for outside direct-blast-effect

										alpha = ( ((tx_x-tar_x) * my_cos_theta + (tx_y-tar_y) * my_sin_theta ) * sqrt_tx_tar + ((rx_x-tar_x) * my_cos_theta + (rx_y-tar_y) * my_sin_theta ) * sqrt_rx_tar )

										#print ("target:",tar_x,tar_y,"angle:",theta,"source:",tx_x,tx_y,"receiver:",rx_x,rx_y,"E-angle:",alpha*180/pi,"TS:",g_cos(alpha))

										if d(tx_x,tx_y,tar_x,tar_y) * d(rx_x,rx_y,tar_x,tar_y) <= (instance.rho_0 + g(alpha, instance))**2: # check for inside range-of-day Cassini oval

											detection_prob[tar_x,tar_y,theta,tx_x,tx_y,rx_x,rx_y] = 1 # sure detection

											#print ("target:",tar_x,tar_y,"angle:",theta,"source:",tx_x,tx_y,"receiver:",rx_x,rx_y,"E-angle:",alpha*180/pi,"TS:",g_cos(alpha))

end_time_coverage = time.time()

print (f"it took {(end_time_coverage - start_time_coverage):.2f} sec to get {len(detection_prob)} detection triples")

In [1]:
import sys
import time
import os
import shutil
import importlib
import random


from math import *
from datetime import datetime


from src.output_func import *
from src.functions import *
from src.classes import *

In [2]:
filename = 'Agadir'

filepath = os.path.join("cfg", filename + '.py')

if os.path.isfile(filepath):

	instance = importlib.import_module("cfg." + filename)
	start_time = time.time()

In [3]:
instance.X


10

In [15]:
def reading_in_ocean_data(instance):

	print(f"reading '" + instance.DIR + instance.INPUT + "'")

	file = open(instance.DIR + "/" + instance.INPUT, "r")
	elevation_data = file.read()
	file.close()

	ncols = int(re.search("ncols (.*)", elevation_data).group(1))
	print(f"number of columns: {ncols}")

	nrows = int(re.search("nrows (.*)", elevation_data).group(1))
	print(f"number of rows: {nrows}")

	latitude = float(re.search("xllcorner (.*)", elevation_data).group(1))
	longitude = float(re.search("yllcorner (.*)", elevation_data).group(1))
	data_delta = float(re.search("cellsize (.*)", elevation_data).group(1))

	elevation_data = re.split("\n+", elevation_data)[6:-1]

	if (nrows != len(elevation_data)+1):

		print(f"Not enough data in file")
		quit()

	map = {}
	ocean = {}
	min_depth = -11022.0
	max_depth = 0.0

	# depth up to 400m in 40m steps
	for z in range(0, 11, 1):

		# for each line in the data
		for y, line_str in enumerate(elevation_data[(nrows - 100 - instance.Y):nrows - 100]):
			
			line_dict = {}
			
			# for each element in the line
			for x, element in enumerate(re.split("\s+", line_str)[:instance.X]):

				element = float(element)

				# if needed, here has to be the code for avg of depths

				# if element < step * -40m
				if element < z*-40:

					ocean[x,y,z] = 1

					if z == 0:

						if element > min_depth and element < 0:

							min_depth = element

						if element < max_depth:

							max_depth = element

						line_dict[x] = element

			if z == 0:
				map[y] = line_dict

	return map, ocean, min_depth, max_depth

In [16]:
map, ocean, min_depth, max_depth = reading_in_ocean_data(instance)

reading 'Instances/Agadir/GMRTv3_4_20171109topo.asc'
number of columns: 696
number of rows: 609
0 -89.50 -90.03 -90.51 -90.90 -91.08 -91.13 -91.10 -90.93 -90.57 -90.04 -89.17 -88.36 -88.08 -87.92 -87.63 -87.25 -86.80 -86.09 -84.88 -83.31 -81.17 -79.23 -78.29 -78.02 -78.50 -79.36 -80.35 -81.24 -81.64 -81.79 -81.87 -81.71 -81.10 -80.29 -79.60 -78.68 -77.29 -75.51 -73.25 -70.71 -67.98 -65.35 -63.12 -61.34 -60.27 -59.56 -58.85 -58.33 -58.13 -58.09 -58.08 -58.11 -58.11 -58.12 -58.13 -58.15 -58.16 -58.17 -58.20 -58.20 -58.11 -57.99 -57.88 -57.73 -57.46 -57.22 -57.16 -57.20 -57.31 -57.49 -57.77 -57.97 -57.83 -57.58 -57.35 -57.17 -57.14 -57.16 -57.16 -57.16 -57.18 -57.17 -57.12 -56.99 -56.63 -56.28 -56.17 -56.12 -56.03 -55.87 -55.52 -55.17 -55.10 -55.02 -54.64 -54.24 -54.09 -53.95 -53.56 -53.18 -53.04 -52.96 -52.81 -52.62 -52.37 -52.16 -52.15 -52.20 -52.21 -52.20 -52.18 -52.14 -52.04 -52.03 -52.36 -52.75 -53.00 -53.15 -53.19 -53.15 -53.06 -52.96 -52.95 -52.87 -52.48 -52.08 -51.95 -51.85 -51.54

In [ ]:
for z in range(0, 11, 1):
    print(z)

In [17]:
ocean

{(0, 0, 0): 1,
 (1, 0, 0): 1,
 (2, 0, 0): 1,
 (3, 0, 0): 1,
 (4, 0, 0): 1,
 (5, 0, 0): 1,
 (6, 0, 0): 1,
 (7, 0, 0): 1,
 (8, 0, 0): 1,
 (9, 0, 0): 1,
 (0, 1, 0): 1,
 (1, 1, 0): 1,
 (2, 1, 0): 1,
 (3, 1, 0): 1,
 (4, 1, 0): 1,
 (5, 1, 0): 1,
 (6, 1, 0): 1,
 (7, 1, 0): 1,
 (8, 1, 0): 1,
 (9, 1, 0): 1,
 (0, 2, 0): 1,
 (1, 2, 0): 1,
 (2, 2, 0): 1,
 (3, 2, 0): 1,
 (4, 2, 0): 1,
 (5, 2, 0): 1,
 (6, 2, 0): 1,
 (7, 2, 0): 1,
 (8, 2, 0): 1,
 (9, 2, 0): 1,
 (0, 3, 0): 1,
 (1, 3, 0): 1,
 (2, 3, 0): 1,
 (3, 3, 0): 1,
 (4, 3, 0): 1,
 (5, 3, 0): 1,
 (6, 3, 0): 1,
 (7, 3, 0): 1,
 (8, 3, 0): 1,
 (9, 3, 0): 1,
 (0, 4, 0): 1,
 (1, 4, 0): 1,
 (2, 4, 0): 1,
 (3, 4, 0): 1,
 (4, 4, 0): 1,
 (5, 4, 0): 1,
 (6, 4, 0): 1,
 (7, 4, 0): 1,
 (8, 4, 0): 1,
 (9, 4, 0): 1,
 (0, 5, 0): 1,
 (1, 5, 0): 1,
 (2, 5, 0): 1,
 (3, 5, 0): 1,
 (4, 5, 0): 1,
 (5, 5, 0): 1,
 (6, 5, 0): 1,
 (7, 5, 0): 1,
 (8, 5, 0): 1,
 (9, 5, 0): 1,
 (0, 6, 0): 1,
 (1, 6, 0): 1,
 (2, 6, 0): 1,
 (3, 6, 0): 1,
 (4, 6, 0): 1,
 (5, 6, 0): 1,
 (6, 6, 0)

In [18]:
map

{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}}

In [19]:
def check_line(x1, y1, z1, x2, y2, z2, ocean):

    """
    Check if a line intersects with any obstacle in the map.

    Ref: https://www.geeksforgeeks.org/bresenhams-algorithm-for-3-d-line-drawing/

    Parameters:
    - x1 (int): x-coordinate of the starting point of the line.
    - y1 (int): y-coordinate of the starting point of the line.
    - z1 (int): z-coordinate of the starting point of the line.
    - x2 (int): x-coordinate of the ending point of the line.
    - y2 (int): y-coordinate of the ending point of the line.
    - z2 (int): z-coordinate of the ending point of the line.
    - ocean (dictonary): 3D dictonary representing ocean

    Returns:
    - int: If the line intersects with an obstacle, returns 1.
    - None: If the line does not intersect with any obstacle.
    """

    ListOfPoints = []
    ListOfPoints.append((x1, y1, z1))

    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)

    if (x2 > x1):
        xs = 1
    else:
        xs = -1
    if (y2 > y1):
        ys = 1
    else:
        ys = -1
    if (z2 > z1):
        zs = 1
    else:
        zs = -1
 
    # Driving axis is X-axis"
    if (dx >= dy and dx >= dz):        
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while (x1 != x2):
            x1 += xs
            if (p1 >= 0):
                y1 += ys
                p1 -= 2 * dx
            if (p2 >= 0):
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz
            ListOfPoints.append((x1, y1, z1))
 
    # Driving axis is Y-axis"
    elif (dy >= dx and dy >= dz):       
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while (y1 != y2):
            y1 += ys
            if (p1 >= 0):
                x1 += xs
                p1 -= 2 * dy
            if (p2 >= 0):
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz
            ListOfPoints.append((x1, y1, z1))
 
    # Driving axis is Z-axis"
    else:        
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while (z1 != z2):
            z1 += zs
            if (p1 >= 0):
                y1 += ys
                p1 -= 2 * dz
            if (p2 >= 0):
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx
            ListOfPoints.append((x1, y1, z1))

    for point in ListOfPoints:

        if point not in ocean:

            return None

    return 1       


In [23]:
answer = check_line(0, 0, 0, 1, 1, 1, ocean)
print(answer)

1


Here we test the target angle attunuation

https://doc.comsol.com/5.6/doc/com.comsol.help.models.aco.submarine_target_strength/submarine_target_strength.html

In [1]:
d_source = 1000             # is the distance from the submarine to the listening point
p_s = 1e-6                  # scattered pressure at the listening point
p_b = 1e-6          

p_in =  abs(p_b)            # the background pressure at the submarine

phi = 90

x = -d_source*cos(phi)
y = -d_source*sin(phi)
z = 0

TS = 20*log10((abs(p_s)/p_in)*d_source/1)





NameError: name 'cos' is not defined

In [18]:
from math import *

x1 = 0
y1 = 0
z1 = 0
x2 = 0
y2 = 0
z2 = 11

In [16]:
sqrt((0.25*x1-0.25*x2)**2 + (0.25*y1-0.25*y2)**2 + (0.0215983*z1-0.0215983*z2)**2)

2.5

In [10]:
sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

1.7320508075688772